# Import packages

In [2]:
import os
import pandas as pd
import numpy as np
import re
import string
import json
# -*- coding:utf-8 -*-

In [2]:
# #  证据等级
# col_name = "证据等级"
# col_ab = "evi"
# que_d = {"1":"Where is pathogen isolated?"}

# # 免疫状态
# col_name = "免疫状态"
# col_ab = "immu"
# que_d = {"1":"What's patients immunity state?"}

# # 人数
# col_name = "样本量"
# col_ab = "num"
# que_d = {"1":"How many patients are ivolved in this study?"}

# # 性别
col_name = "患者性别"
col_ab = "gender"
que_d = {"1":"What's gender of the patient?"}

# # # 年龄
# col_name = "年龄"
# col_ab = "age"
# que_d = {"1":"How old is patient?"}

# 将标注文档转成excel

In [ ]:
input_path = "input"
sped_l = [ele for ele in os.listdir(input_path) if "." not in ele]
ent = ["是否是致病菌", "样本量", "患者性别", "年龄", "免疫状态", "肺部基础疾病", "证据等级", "预后"]

for sped in sped_l:

    collection = {}
    paper_l = [
        ele for ele in os.listdir("%s/%s/outputs" % (input_path, sped))
        if "json" in ele
    ]

    for paper in paper_l:

        pattern = {
            "物种名称": "",
            "是否是致病菌": "",
            "样本量": "",
            "患者性别": "",
            "年龄": "",
            "免疫状态": "",
            "肺部基础疾病": "",
            "证据等级": "",
            "预后": ""
        }

        with open("%s/%s/outputs/%s" % (input_path, sped, paper),
                  "r",
                  encoding='UTF-8') as f:
            ano = json.load(f)

        evi_list = []
        evis_list = []

        if ano["outputs"] != {}:

            for i in ano["outputs"]["annotation"]["A"]:

                if len(i) > 1:

                    # 保存所有信息到pattern
                    if i["name"] in ent:
                        pattern[i["name"]] = i["value"]

                    # 获取标注的是否是病原菌的信息
                    if i["name"]  in col_name:
                        evi_list.append(i["value"])

            # j为原文本
            for j in ano["outputs"]["annotation"]["T"]:

                if j is not None:

                    if len(j) > 1:

                        # 获取标注的判断病原菌的文本
                        if j["name"] == "物种":
                            
                            pattern["物种名称"] = j["value"]

                        # 获取标注的判断证据等级的文本
                        if j["name"] in col_name:

                            # 比较等级
                            evis_list.append(j["value"])

                            if len(evis_list) > 1:

                                evi_new = evi_list[len(evis_list) - 1]
                                evi_old = evi_list[len(evis_list) - 2]

                                if evi_new < evi_old:

                                    pattern[col_name] == j["value"]
                                    anr = ano["content"].replace("\r\n", "  ")
                                    # 获取证据等级所在的整个句子
                                    new_start = [
                                        anr.rfind(
                                            "%s" % k, 0, j["start"])
                                        for k in string.punctuation
                                        if anr.rfind(
                                            "%s" % k, 0, j["start"]) > 0
                                    ]
                                    new_start.append(0)
                                    new_start = np.max(new_start)
                                    new_end = [
                                        anr.find(
                                            "%s" % k, j["end"], -1)
                                        for k in string.punctuation
                                        if anr.find(
                                            "%s" % k, j["end"], -1) > 0
                                    ]
                                    new_end.append(len(anr) - 1)
                                    new_end = np.min(new_end)
                                    evi_txt = anr[new_start +
                                                            1:new_end + 1]
                                    pattern["answer"] = j["value"]
                                    pattern["old_start"] = j["start"]
                                    pattern["old_end"] = j["end"]
                                    pattern["sup_evi"] = evi_txt
                                    pattern["new_start"] = new_start + 1
                                    pattern["new_end"] = new_end + 1

                            else:
                                
                                anr = ano["content"].replace("\r\n", "  ")
                                new_start = [
                                        anr.rfind(
                                            "%s" % k, 0, j["start"])
                                        for k in string.punctuation
                                        if anr.rfind(
                                            "%s" % k, 0, j["start"]) > 0
                                    ]
                                new_start.append(0)
                                new_start = np.max(new_start)

                                new_end = [
                                    anr.find(
                                        "%s" % k, j["end"], -1)
                                    for k in string.punctuation
                                    if anr.find(
                                        "%s" % k, j["end"], -1) > 0
                                ]
                                new_end.append(len(anr) - 1)
                                new_end = np.min(new_end)
                                evi_txt = anr[new_start +
                                                        1:new_end + 1]
                                pattern["answer"] = j["value"]
                                pattern["old_start"] = j["start"]
                                pattern["old_end"] = j["end"]
                                pattern["sup_evi"] = evi_txt
                                pattern["new_start"] = new_start + 1
                                pattern["new_end"] = new_end + 1
                            
        pattern["full"] = ano["content"]
        pattern["id"] = 'https://pubmed.ncbi.nlm.nih.gov/%s/' % paper.replace(
                ".json", "")

        collection[paper.replace(".json", "")] = pattern

    df = pd.DataFrame(collection).T
    print(df.shape)
    df.to_excel("文献标注/%s_%s.xls" % (sped, col_ab))


# 添加支持证据

In [ ]:
add_list = [ele for ele in os.listdir("文献标注-校对") if "_add" in ele]
ori_list = [ele for ele in os.listdir("文献标注") if col_ab in ele]

for add in add_list:

    df = pd.read_excel("文献标注-校对/%s" % add, header=0, index_col=0)
    
    # 读取两个人标注的文档并保存

    dfo_l = []

    for ori in ori_list:

        if ori.split("-")[0] in add:

            dfo = pd.read_excel("文献标注/%s" % ori, header=0, index_col=0)
            dfo_l.append(dfo)

            if "sup_evi" in dfo.columns:
                
                for idx in dfo.index: 
                        
                        if pd.isna(dfo.loc[idx, "sup_evi"]) is True:

                            dfo.loc[idx, "answer"] =  dfo_l[0].loc[idx, "answer"]
                            dfo.loc[idx, "old_start"] = dfo_l[0].loc[idx, "old_start"]
                            dfo.loc[idx, "old_end"] = dfo_l[0].loc[idx, "old_end"]
                            dfo.loc[idx, "sup_evi"] = dfo_l[0].loc[idx, "sup_evi"]
                            dfo.loc[idx, "new_start"] = dfo_l[0].loc[idx, "new_start"]
                            dfo.loc[idx, "new_end"] = dfo_l[0].loc[idx, "new_end"]

                df["sup_evi"] = dfo["sup_evi"]
                df["old_start"] = dfo["old_start"]
                df["answer"] = dfo["answer"]
                df["full"] = dfo["full"]
    df.to_excel("文献标注-校对/%s" % add.replace("add", "%s" % col_ab))

In [30]:
var_l = ["num", "age", "gender", "immu", "evi"]
dfv_l = []
for var in var_l:
    file_list = [ele for ele in os.listdir("文献标注-校对") if "_%s" % var in ele]
    df_l = []
    for file in file_list:
        df = pd.read_excel("文献标注-校对/%s" % file, header=0)
        df_l.append(df)
    dfv = pd.concat(df_l)
    dfv["%s_origin" % var] = dfv["answer"]
    dfv_l.append(dfv[["%s_origin" % var]])
df0 = dfv[['物种名称', '是否是致病菌', '样本量', '患者性别', '年龄', '免疫状态', '证据等级', 'ID']]
df1 = pd.concat(dfv_l, 1)
dfh = pd.concat((df0, df1), 1)
link_l = ["https://pubmed.ncbi.nlm.nih.gov/%s/" % ele for ele in dfh['ID']]
dfh.index = range(dfh.shape[0])
dfh.index = [dfh["物种名称"][i].replace(" ", "_") + "_" + str(dfh["ID"][i]) for i in range(dfh.shape[0])]
dfh["ID"] = link_l
dfh['预测概率'] = "[1]"
dfh['标注方式'] = '人工标注'


In [31]:

dfh.columns = ['物种名称', '是否是致病菌', '样本量', '患者性别', '年龄', '免疫状态', '证据等级','原文链接', '样本量文本',
       '年龄文本', '性别文本', '免疫文本', '证据文本',  '预测概率','标注方式']

dfh = dfh[['物种名称', '是否是致病菌', '预测概率', '样本量', '样本量文本','年龄', '年龄文本',
           '患者性别', '性别文本', '免疫状态', '免疫文本', '证据等级', '证据文本', '原文链接', '标注方式']]
dfh[dfh.isna()]= "NA"
dfh.loc[dfh['是否是致病菌']== "是", "是否是致病菌"] = "yes"
dfh.loc[dfh['是否是致病菌'] != "yes", "是否是致病菌"] = "no"
dfh.to_csv("../7.Result_Analysis/human_label.csv")